In [1]:
%run ./sentiment_data.ipynb

In [2]:
data

,full_text
0,\n This is a very good tin.\n
1,\n I can't tell what I hate about this mic ...
2,\n I was worried at first but since deliver...
3,\n Extremely disappointed that this sat on ...
4,\n Would love to return it if I could but i...
5,\n The fact it doesn't have it's own power ...
6,\n Finally found a good mixer and volume co...
7,\n Okay first off I am no audio professiona...
8,\n A perfect stand for the GOXLR it prevent...
9,\n WORTH THE PRICE!!!!! Hands down best mix...


In [2]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('vader_lexicon')
from nltk.corpus import stopwords
from tqdm import tqdm

import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dalmas.otieno/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/dalmas.otieno/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [5]:
def decontractions(phrase: str)->str:
    map_list = [
        ["won't", "will not"],
        ["can't", "can not"],
        ["can’t", "can not"],
        
        # general
        ["n't", " not"],
        ["'re", " are"],
        ["'s", " is"],
        ["'d", " would"],
        ["'ll", " will"],
        ["'t", " not"],
        ["'ve", " have"],
        ["'m", " am"],
        ["n’t", " not"],
        ["’re", " are"],
        ["’s", " is"],
        ["’d", " would"],
        ["’ll", " will"],
        ["’t", " not"],
        ["’ve", " have"],
        ["’m", " am"]
    ]
    for amap in map_list:
        phrase = re.sub(amap[0], amap[1], phrase)
    return phrase
# test
decontractions("I can't I am done. Whatever you are doing won't work.")

'I can not I am done. Whatever you are doing will not work.'

In [6]:
def preprocess(series_column: pd.Series, remove_stopwords: bool) -> list:
    preprocessed = []
    characters_to_remove = ['\\r','\\"','\\n']
    for sentence in tqdm(series_column.values):
        sentence = str(sentence)
        for char in characters_to_remove:
            sentence = sentence.replace(char, ' ')
        sentence = re.sub('[^A-Za-z0-9]+', ' ', sentence)
        sentence = decontractions(sentence)
        
        if remove_stopwords:
            sentence = ' '.join(sent for sent in sentence.split() if sent not in stopwords.words())
        preprocessed.append(sentence.lower().strip())
    return preprocessed

In [9]:
data['clean_text'] = preprocess(data['full_text'], False)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 9517.53it/s]


In [10]:
data

,full_text,clean_text
0,\n This is a very good tin.\n,this is a very good tin
1,\n I can't tell what I hate about this mic ...,i can t tell what i hate about this mic but i ...
2,\n I was worried at first but since deliver...,i was worried at first but since delivery i li...
3,\n Extremely disappointed that this sat on ...,extremely disappointed that this sat on my des...
4,\n Would love to return it if I could but i...,would love to return it if i could but it is t...
5,\n The fact it doesn't have it's own power ...,the fact it doesn t have it s own power leads ...
6,\n Finally found a good mixer and volume co...,finally found a good mixer and volume control ...
7,\n Okay first off I am no audio professiona...,okay first off i am no audio professional by a...
8,\n A perfect stand for the GOXLR it prevent...,a perfect stand for the goxlr it prevents 99 o...
9,\n WORTH THE PRICE!!!!! Hands down best mix...,worth the price hands down best mixer for any ...


In [14]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def generate_sentiment_scores(df: pd.DataFrame):
    def overall_sentiment(negative: float, positive: float)->str:
        if negative > positive:
            return 'negative'
        elif positive > negative:
            return 'positive'
        return 'neutral'
    sid = SentimentIntensityAnalyzer()
    neg, pos, overall = [], [], []
    for sentence in tqdm(df['clean_text'].values):
        sentence_sentiment_score = sid.polarity_scores(sentence)
        neg.append(sentence_sentiment_score['neg'])
        pos.append(sentence_sentiment_score['pos'])
        overall.append(overall_sentiment(sentence_sentiment_score['neg'], sentence_sentiment_score['pos']))
    return neg, pos, overall

In [15]:
data['negative'], data['positive'], data['overall'] = generate_sentiment_scores(data)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 2889.56it/s]


In [16]:
data

,full_text,clean_text,negative,positive,overall
0,\n This is a very good tin.\n,this is a very good tin,0.000,0.444,positive
1,\n I can't tell what I hate about this mic ...,i can t tell what i hate about this mic but i ...,0.390,0.000,negative
2,\n I was worried at first but since deliver...,i was worried at first but since delivery i li...,0.135,0.274,positive
3,\n Extremely disappointed that this sat on ...,extremely disappointed that this sat on my des...,0.274,0.000,negative
4,\n Would love to return it if I could but i...,would love to return it if i could but it is t...,0.000,0.140,positive
5,\n The fact it doesn't have it's own power ...,the fact it doesn t have it s own power leads ...,0.078,0.000,negative
6,\n Finally found a good mixer and volume co...,finally found a good mixer and volume control ...,0.000,0.244,positive
7,\n Okay first off I am no audio professiona...,okay first off i am no audio professional by a...,0.071,0.129,positive
8,\n A perfect stand for the GOXLR it prevent...,a perfect stand for the goxlr it prevents 99 o...,0.126,0.202,positive
9,\n WORTH THE PRICE!!!!! Hands down best mix...,worth the price hands down best mixer for any ...,0.000,0.295,positive


In [3]:
data = pd.DataFrame({'Name': ['Nick', 'Matthew', 'Paul'],
                     'Text': ["Lived in Norway, England, Spain and Germany with his car",
                              "Used his bikes in England. Loved his bike",
                              "Lived in Alaska"]})

In [4]:
data

,Name,Text
0,Nick,"Lived in Norway, England, Spain and Germany wi..."
1,Matthew,Used his bikes in England. Loved his bike
2,Paul,Lived in Alaska


In [5]:

def search_terms(text: str, example_list: list)->str:
    example_list = [i.lower() for i in example_list]
    text_list = text.split()
    extracted_example = [t for t in text_list if t.lower() in example_list]
    
    if len(extracted_example)>0:
        return ' '.join(extracted_example)
    return None

In [ ]:
EXAMPLE_LIST = ["England", "Bike"]
data['Text'].apply(lambda text: search_terms(text, EXAMPLE_LIST))